In [ ]:
from qTools import *

import numpy as np
from datetime import datetime
import os
import qutip

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

## Drive coupling extension 
### (To be put together later)

## System Parameters

In [ ]:
omega_q     = 10
omega_drive = 10 + 1
omega_Rabi  = 10

detuning = omega_q - omega_drive

## Creating Qubit and Drive

In [ ]:
qubit = Qubit(frequency=omega_q)

#drive terms
# drive_term1 = qubit.addTerm(sigmam, omega_Rabi)
# drive_term2 = qubit.addTerm(sigmap, omega_Rabi)

In [ ]:
qubit.initialState = {0:0.5, 1:0.5}
# qubit.initialState = qubit.initialState * (np.exp(-1j*0.3))

In [ ]:
print(qubit.initialState.A)

## (For time dependency)

In [ ]:
drive_coeff = lambda t: np.exp(-1j * detuning * t)

tSweep = qubit.simulation.timeDependency.createSweep(system=qubit)

def coeff_sweep(sweep):
    qubit = sweep.system
    time = qubit.simulation.stepSize * sweep.index
    
    qubit.subSys['Qubit2'].frequency = np.conj(drive_coeff(time))
    qubit.subSys['Qubit3'].frequency = drive_coeff(time)

tSweep.sweepFunction = coeff_sweep

## Define Simulation Output

In [ ]:
def expect(sim, state):
    # Fidelity to intial
    sim.qRes.results['fidelity'].append(fidelityKet(state, qubit.initialState))

    # Bloch Sphere coordinates
    sim.qRes.results['x'].append(expectationKet(sigmax(), state))
    sim.qRes.results['y'].append(expectationKet(sigmay(), state))
    sim.qRes.results['z'].append(expectationKet(sigmaz(), state))

qubit.simulation.compute = expect

## Simulation Parameters and Sweeps

In [ ]:
qubit.simulation.totalTime = 0.1
qubit.simulation.stepSize = 0.01
qubit.simulation.delStates = True
print(qubit.simulation.stepCount)

In [ ]:
detuning_sweep = qubit.simulation.Sweep.createSweep(system=qubit, sweepKey='frequency', sweepList=np.linspace(0, 20, 5))

## Run Simulation

In [ ]:
qubit.simulation.run()

## Plotting

In [ ]:
# plt.pcolormesh(np.arange(0, qSim.totalTime + qSim.stepSize, qSim.stepSize), wSweep.sweepList, qSim.qRes.results['fidelity'])
# plt.xlabel('Time (μs)')
# plt.ylabel('Frequency (MHz)')
# plt.show()

In [ ]:
x = qubit.simulation.qRes.results['x']
y = qubit.simulation.qRes.results['y']
z = qubit.simulation.qRes.results['z']

t = np.arange(0, qubit.simulation.totalTime + qubit.simulation.stepSize, qubit.simulation.stepSize)

plt.plot(x, c='b')
plt.plot(y, c='r')
plt.plot(z, c='g')
plt.legend(['x', 'y', 'z'])
plt.show()

In [ ]:
x = qubit.simulation.qRes.results['x']
y = qubit.simulation.qRes.results['y']
z = qubit.simulation.qRes.results['z']

plt.plot(x, y)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.show()

In [ ]:
#If no normal sweeps have been done

x = qubit.simulation.qRes.results['x']
y = qubit.simulation.qRes.results['y']
z = qubit.simulation.qRes.results['z']

b = qutip.Bloch3d()
b.add_points([x, y, z])
b.show()

In [ ]:
#If a normal sweep has been done

x = qubit.simulation.qRes.results['x']
y = qubit.simulation.qRes.results['y']
z = qubit.simulation.qRes.results['z']

for xi, yi, zi in zip(x, y, z):
    b = qutip.Bloch()
    b.add_points([xi, yi, zi])
    b.show()